## 六. 代码理解(Code Understanding)

代码理解用到的工具和文档问答差不多，不过我们的输入是一个项目的代码。

- Co-Pilot-esque functionality that can help answer questions from a specific library, help you generate new code

In [3]:
import sys
sys.path.append("../")
from models import azure_chat_model, azure_embeddings


In [5]:
# Helper to read local files
import os

# Vector Support
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings

# Model and chain
from langchain.chat_models import ChatOpenAI

# Text splitters
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader


In [6]:

root_dir = '../data/thefuzz/'
docs = []

# Go through each folder
for dirpath, dirnames, filenames in os.walk(root_dir):
    
    # Go through each file
    for file in filenames:
        try: 
            # Load up the file as a doc and split
            loader = TextLoader(os.path.join(dirpath, file), encoding='utf-8')
            docs.extend(loader.load_and_split())
        except Exception as e: 
            pass

In [7]:
print (f"You have {len(docs)} documents\n")
print ("------ Start Document ------")
print (docs[0].page_content[:300])


You have 170 documents

------ Start Document ------
from timeit import timeit
import math
import csv

iterations = 100000


reader = csv.DictReader(open('data/titledata.csv'), delimiter='|')
titles = [i['custom_title'] for i in reader]
title_blob = '\n'.join(titles)


cirque_strings = [
    "cirque du soleil - zarkana - las vegas",
    "cirque du sol


In [9]:
docsearch = FAISS.from_documents(docs, azure_embeddings)

In [10]:
from langchain.chains import RetrievalQA
# Get our retriever ready
qa = RetrievalQA.from_chain_type(llm=azure_chat_model, chain_type="stuff", retriever=docsearch.as_retriever())

In [12]:
query = "What function do I use if I want to find the most similar item in a list of items?"
#query = "What function do I use if I want to find the single best match above a score in a list of choices"
output = qa.run(query)
output

'You can use the `process.extractOne()` function from the `thefuzz` library to find the most similar item in a list of items. Here\'s an example:\n\n```python\nfrom thefuzz import process\n\nchoices = ["item1", "item2", "item3", "item4"]\nquery = "query"\n\nbest_match = process.extractOne(query, choices)\n```\n\nThe `extractOne()` function returns a tuple containing the best match and its similarity score. In this example, `best_match` will be a tuple like `("item2", 86)`, indicating that "item2" is the best match with a similarity score of 86.'

In [15]:
query = "Can you write the code to use the process.extractOne() function? Only respond with code. No other text or explanation"
output = qa.run(query)
print(output)

best = process.extractOne(query, choices)
